In [ ]:
# ==========================================
# Odd vs Even CNN Classifier (from MNIST)
# ==========================================

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import time, random

# ==============================
# 1️⃣ Load MNIST Dataset
# ==============================
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#C:\Users\<your_username>\.keras\datasets\mnist.npz
#~/.keras/datasets/mnist.npz

print("Original MNIST:")
print("Train:", x_train.shape, y_train.shape)
print("Test :", x_test.shape, y_test.shape)


# ==============================
# 2️⃣ Convert digits → Odd/Even
# ==============================
# Even -> 0, Odd -> 1
y_train = np.array([0 if d % 2 == 0 else 1 for d in y_train])
y_test  = np.array([0 if d % 2 == 0 else 1 for d in y_test])

class_labels = ['Even', 'Odd']


# ==============================
# 3️⃣ Preprocess Data
# ==============================
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# Add channel dimension
x_train = np.expand_dims(x_train, -1)
x_test  = np.expand_dims(x_test, -1)

# One-hot encoding
y_train_cat = to_categorical(y_train, 2)
y_test_cat  = to_categorical(y_test, 2)

# Train/validation split
x_train, x_val, y_train_cat, y_val_cat = train_test_split(
    x_train, y_train_cat, test_size=0.2, random_state=42
)


# ==============================
# 4️⃣ CNN Model Definition
# ==============================
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


# ==============================
# 5️⃣ Training
# ==============================
start_time = time.time()

history = model.fit(
    x_train, y_train_cat,
    epochs=10,
    batch_size=64,
    validation_data=(x_val, y_val_cat)
)

train_time = time.time() - start_time
print(f"\n✅ Training completed in {train_time:.2f} sec")


# ==============================
# 6️⃣ Evaluate
# ==============================
start_test = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test_cat)

test_time = time.time() - start_test

print(f"\n🧮 Testing time: {test_time:.2f} sec")
print(f"🎯 Test Accuracy: {test_acc*100:.2f}%")


# ==============================
# 7️⃣ Plot Accuracy & Loss
# ==============================
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curve')
plt.savefig("odd_even_accuracy.png")
plt.show()


plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')
plt.savefig("odd_even_loss.png")
plt.show()


# ==============================
# 8️⃣ Predict 10 Random Images
# ==============================
indices = random.sample(range(len(x_test)), 10)

plt.figure(figsize=(15,8))
correct = 0

for i, idx in enumerate(indices):

    img = x_test[idx]
    true_label = class_labels[y_test[idx]]

    pred = model.predict(np.expand_dims(img,0), verbose=0)
    pred_class = class_labels[np.argmax(pred)]
    conf = np.max(pred)

    if pred_class == true_label:
        correct += 1

    plt.subplot(2,5,i+1)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(f"P:{pred_class}\nT:{true_label}\n{conf:.2f}")
    plt.axis('off')

plt.tight_layout()
plt.show()

print(f"\n✅ Correct: {correct}/10 ({correct*10}%)")


# ==============================
# 9️⃣ Save Model
# ==============================
model.save("mnist_odd_even_cnn.h5")
print("\n💾 Model saved as 'mnist_odd_even_cnn.h5'")


ModuleNotFoundError: No module named 'tensorflow'